In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, M2M100ForConditionalGeneration, M2M100Tokenizer

model_name = "facebook/wmt21-dense-24-wide-en-x"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
model = model.cuda()
model = model.half()
model = model.eval()
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
import time

dataset = load_dataset("databricks/databricks-dolly-15k")

Found cached dataset json (/root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-6e0f9ea7eaa0ee08/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 420.31it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

### base translation

In [4]:
def translate(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.get_lang_id("ru"),
        max_new_tokens=1024,
    )
    result = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[
        0
    ]
    return result

In [5]:
import torch

In [7]:
with torch.no_grad():
    start = time.time()
    fields = [
        'instruction', 'context', 'response'
    ]
    for item in dataset['train'].select(range(5)):
        # print(item)
        for field in fields:
            text = item[field]
            result = translate(
                model=model,
                tokenizer=tokenizer,
                text=text
            )
        # print(result)
    print(time.time() - start)

29.696838855743408


### tensor rt optimizations

In [4]:
import torch

example = tokenizer("hello world", return_tensors='pt').to('cuda')
example['decoder_input_ids'] = example['input_ids']

# input_ids: Optional[torch.LongTensor] = None,
# attention_mask: Optional[torch.Tensor] = None,
# decoder_input_ids: Optional[torch.LongTensor] = None,
# decoder_attention_mask: Optional[torch.LongTensor] = None,
# head_mask: Optional[torch.Tensor] = None,
# decoder_head_mask: Optional[torch.Tensor] = None,
# cross_attn_head_mask: Optional[torch.Tensor] = None,
# encoder_outputs: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
# past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
# inputs_embeds: Optional[torch.FloatTensor] = None,
# decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
# labels: Optional[torch.LongTensor] = None,

traced_model = torch.jit.trace(model, 
	example_kwarg_inputs={
		"input_ids": torch.tensor([[1]]),
		"attention_mask": torch.tensor([[1]]),
		"decoder_input_ids": torch.tensor([[1]]),
		"decoder_attention_mask": torch.tensor([[1]]),
		"head_mask": torch.tensor([[1]]),
		"decoder_head_mask": torch.tensor([[1]]),
		"cross_attn_head_mask": torch.tensor([[1]]),
		"encoder_outputs": torch.tensor([[1]]),
		"past_key_values": torch.tensor([
      		[[[1]]]
        ]),
		"inputs_embeds": torch.tensor([[1]]),
		# "decoder_inputs_embeds": torch.tensor([[1]]),
		"labels": torch.tensor([[1]]),
	},
 	strict=False
)

tensor([[[[1]]]])


IndexError: tuple index out of range

{'input_ids': [128001, 97394, 1211, 2],
 'attention_mask': [1, 1, 1, 1],
 'decoder_input_ids': [128001, 97394, 1211, 2]}

In [3]:
[[[1]]][0][0]

[1]

In [ ]:
"""
Found Tensor and Tuple[Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor], Tuple[Tensor, Tensor, Tensor, Tensor]]
"""